# Preprocessing Nebraska Allocation data for WaDEQA upload.
- Date Updated: 08/26/2020
- Purpose:  To preprocess the Nebraska data into one master file for simple DataFrame creation and extraction.
- Joining API surface water data - to POD shapefile data via **RightID**.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
#Working Directory and Input Files
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Nebraska/WaterAllocation/RawInputData"
os.chdir(workingDir)

In [3]:
#Retrieve POD shapefile data
sfInput = "ActiveSWPODs08242020_input.csv"
dfpodsftemp = pd.read_csv(sfInput)
dfpodsf = dfpodsftemp[['RightID', 'HUC_12', 'Longitude', 'Latitude']] #columns of interest
dfpodsf

,RightID,HUC_12,Longitude,Latitude
0,933,1.025000e+11,-101.964731,40.033267
1,945,1.025000e+11,-101.816246,40.035814
2,1501,1.025002e+11,-99.410350,40.005894
3,1504,1.025002e+11,-99.404383,40.009103
4,1505,1.025002e+11,-99.375045,40.013726
...,...,...,...,...
8786,3340,1.027020e+11,-97.146450,40.719856
8787,3453,1.027020e+11,-96.946811,40.474404
8788,13849,1.015001e+11,-98.484968,42.704781
8789,13850,1.024001e+11,-95.973326,40.086231


In [4]:
# Get all surface water points from NeDNR API.
# Note: API has lots of NULL values, have to put a hard stop of what to search.
# May get duplicates, should be roughly 120,000 good records to work with (according to NE).
dfsw = pd.DataFrame()
page = 1


# for i in range(length):
while page < 20:
    url = "https://nednr.nebraska.gov/IwipApi/api/v1/WaterRights/AllSurfaceWaterPoints?page=" + str(page)
    page = 1 + page
    responseD = json.loads(requests.get(url).text)
    DLtL = responseD['Results']
    length = len(DLtL)
    for i in range(length):
        row = pd.DataFrame([DLtL[i]])
        dfsw = dfsw.append(row)

dfapisw = dfsw.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
print(len(dfapisw))
dfapisw

12562


,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203.0,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None
0,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203.0,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None
0,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203.0,1975-08-14T00:00:00,None,None,None,None,None,None,None
0,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203.0,1975-08-14T00:00:00,None,None,None,None,None,None,None
0,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0.0,AF,0,None,314.0,Jefferson,Little Blue,102702070205.0,1969-06-25T00:00:00,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,13057,Active,IR,A-19389,NENW S17-T3-R13E,16174,1,1,1F,None,Pump,None,Long Branch,0.43,CFS,193,70,29.6,Richardson,Nemaha,102400080205.0,2015-08-28T00:00:00,2018-09-01T00:00:00,2015-12-21T00:00:00,None,Open,2019-04-23T00:00:00,Open,Natural Flow (Reminder)
0,13852,Active,IR,A-19740,NENW S17-T3-R13E,17868,1,1,1F,None,None,None,Long Branch,0.32,CFS,None,70,22.1,Richardson,Nemaha,102400080205.0,2021-01-14T00:00:00,2023-09-01T00:00:00,2021-05-27T00:00:00,None,None,None,None,None
0,1043,Active,IR,A-1285,NW S7-T5-R35W,2380,7,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404.0,1913-04-28T00:00:00,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder)
0,1043,Active,IR,A-1285,NW S7-T5-R35W,2382,1,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404.0,1913-04-28T00:00:00,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder)


In [5]:
#Merging dataframes into one, using left-join.
df = pd.merge(dfapisw, dfpodsf, left_on='RightID', right_on='RightID', how='inner')
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203.0,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203.0,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203.0,1975-08-14T00:00:00,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203.0,1975-08-14T00:00:00,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0.0,AF,0,None,314.0,Jefferson,Little Blue,102702070205.0,1969-06-25T00:00:00,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12236,542,Active,IU,D-680,SENE S11-T11-R26W,12280,1,1,1A,None,Six Mile Canal,None,Platte River,23.8,CFS,10681,70,1666.0,Lincoln,Twin Platte,102001010408.0,1894-10-22T00:00:00,None,1899-09-01T00:00:00,None,Open,2020-09-15T00:00:00,Open,Natural Flow (Reminder),1.020010e+11,-100.250737,40.937344
12237,13057,Active,IR,A-19389,NENW S17-T3-R13E,16174,1,1,1F,None,Pump,None,Long Branch,0.43,CFS,193,70,29.6,Richardson,Nemaha,102400080205.0,2015-08-28T00:00:00,2018-09-01T00:00:00,2015-12-21T00:00:00,None,Open,2019-04-23T00:00:00,Open,Natural Flow (Reminder),1.024001e+11,-95.987384,40.229258
12238,1043,Active,IR,A-1285,NW S7-T5-R35W,2380,7,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404.0,1913-04-28T00:00:00,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094
12239,1043,Active,IR,A-1285,NW S7-T5-R35W,2382,1,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404.0,1913-04-28T00:00:00,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094


In [6]:
# Changing datatype of used date fields. 
df['PriorityDate'] = pd.to_datetime(df['PriorityDate'], errors = 'coerce')
df['PriorityDate'] = pd.to_datetime(df["PriorityDate"].dt.strftime('%m/%d/%Y'))
df.head(3)

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203.0,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203.0,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203.0,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979


In [7]:
# Making Sure datatype of HUC12 is int.
def assignHUC12(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        outList = int(colrowValue)
    return outList

df['HUC12'] = df.apply(lambda row: assignHUC12(row['HUC12']), axis=1)
df.head(3)

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979


In [8]:
#Creating Beneficial Use from NeDWR Provdied code.  See metaData_SurfaceWaterWebSimpleSearch.pdf for details.

NebraskaBenUseCodeDict = {
"CO" : "Cooling",
"DG" : "Dredge",
"DI" : "Domestic, Irrigation and Manufacturing",
"DO" : "Domestic",
"DS" : "Domestic Storage",
"FC" : "Fish Culture",
"FL" : "Flood Control",
"FW" : "Fish and Wildlife",
"IF" : "Instream Flow",
"IG" : "Induced Ground Water Recharge",
"IN" : "Intentional Underground Storage",
"IR" : "Irrigation from Natural Stream",
"IS" : "Irrigation and Storage (an appropriation approved for both uses)",
"IU" : "Irrigation and Incidental Underground Storage",
"MF" : "Manufacturing",
"ML" : "Maintain Level of a Lake",
"MU" : "Municipal",
"OU" : "Incidental Underground Storage",
"PI" : "Power and Incidental Underground Storage",
"PR" : "Power",
"PS" : "Supplemental Power and Incidental Underground Storage",
"PW" : "Public Water Supply",
"RC" : "Groundwater Recharge",
"RD" : "Raise Dam (for increase in head for power production)",
"SC" : "Supplemental Cooling",
"SD" : "Supplemental Domestic",
"SF" : "Supplemental Fish Culture",
"SI" : "Supplemental Irrigation",
"SO" : "Storage Use-only",
"SP" : "Supplemental Power",
"SS" : "Supplemental Storage",
"ST" : "Storage",
"SU" : "Storage and Incidental Underground Storage",
"TI" : "Temporary Transfer to In-Stream Use",
"UI" : "Supplemental Irrigation and Incidental Underground Storage",
"US" : "Incidental Underground Storage",
"WS" : "Waste Storage",
"WT" : "Wetlands"}

def assignRightUse(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = NebraskaBenUseCodeDict[String1]
        except:
            outList = "Unspecified"
    return outList

df['BeneficialUseCategory'] = df.apply(lambda row: assignRightUse(row['RightUse']), axis=1)
df.head(3)

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only


In [9]:
# AllocationFlow_CFS - based on reporeted Unit
def assignAllocationFlow_CFS(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "CFS":
            outList = colvA
        else:
            outList = ''
    return outList

df['AllocationFlow_CFS'] = df.apply(lambda row: assignAllocationFlow_CFS(row['ProGrant'], row['Units']), axis=1)
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory,AllocationFlow_CFS
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only,
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only,
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0.0,AF,0,None,314.0,Jefferson,Little Blue,102702070205,1969-06-25,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120,Storage Use-only,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12236,542,Active,IU,D-680,SENE S11-T11-R26W,12280,1,1,1A,None,Six Mile Canal,None,Platte River,23.8,CFS,10681,70,1666.0,Lincoln,Twin Platte,102001010408,1894-10-22,None,1899-09-01T00:00:00,None,Open,2020-09-15T00:00:00,Open,Natural Flow (Reminder),1.020010e+11,-100.250737,40.937344,Irrigation and Incidental Underground Storage,23.8
12237,13057,Active,IR,A-19389,NENW S17-T3-R13E,16174,1,1,1F,None,Pump,None,Long Branch,0.43,CFS,193,70,29.6,Richardson,Nemaha,102400080205,2015-08-28,2018-09-01T00:00:00,2015-12-21T00:00:00,None,Open,2019-04-23T00:00:00,Open,Natural Flow (Reminder),1.024001e+11,-95.987384,40.229258,Irrigation from Natural Stream,0.43
12238,1043,Active,IR,A-1285,NW S7-T5-R35W,2380,7,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404,1913-04-28,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094,Irrigation from Natural Stream,2.67
12239,1043,Active,IR,A-1285,NW S7-T5-R35W,2382,1,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404,1913-04-28,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094,Irrigation from Natural Stream,2.67


In [10]:
# AllocationVolume_AF - based on reporeted Unit
def assignAllocationVolume_AF(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "AF":
            outList = colvA
        else:
            outList = ''
    return outList

df['AllocationVolume_AF'] = df.apply(lambda row: assignAllocationVolume_AF(row['ProGrant'], row['Units']), axis=1)
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory,AllocationFlow_CFS,AllocationVolume_AF
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only,,0.0
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only,,0.0
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,,0.0
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,,0.0
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0.0,AF,0,None,314.0,Jefferson,Little Blue,102702070205,1969-06-25,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120,Storage Use-only,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12236,542,Active,IU,D-680,SENE S11-T11-R26W,12280,1,1,1A,None,Six Mile Canal,None,Platte River,23.8,CFS,10681,70,1666.0,Lincoln,Twin Platte,102001010408,1894-10-22,None,1899-09-01T00:00:00,None,Open,2020-09-15T00:00:00,Open,Natural Flow (Reminder),1.020010e+11,-100.250737,40.937344,Irrigation and Incidental Underground Storage,23.8,
12237,13057,Active,IR,A-19389,NENW S17-T3-R13E,16174,1,1,1F,None,Pump,None,Long Branch,0.43,CFS,193,70,29.6,Richardson,Nemaha,102400080205,2015-08-28,2018-09-01T00:00:00,2015-12-21T00:00:00,None,Open,2019-04-23T00:00:00,Open,Natural Flow (Reminder),1.024001e+11,-95.987384,40.229258,Irrigation from Natural Stream,0.43,
12238,1043,Active,IR,A-1285,NW S7-T5-R35W,2380,7,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404,1913-04-28,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094,Irrigation from Natural Stream,2.67,
12239,1043,Active,IR,A-1285,NW S7-T5-R35W,2382,1,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404,1913-04-28,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094,Irrigation from Natural Stream,2.67,


In [11]:
# For creating WaterSourceName
def assignWaterSourceName(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        outList = colrowValue.strip()
    return outList

df['in_WaterSourceName'] = df.apply(lambda row: assignWaterSourceName(row['SourceName']), axis=1)

## WaDE Custom Elements (due to missing state site info)

In [12]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDENE_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = df['in_WaterSourceName']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

df['in_WaterSourceNativeID'] = df.apply(lambda row: retrieveWaterSourceNativeID(row['in_WaterSourceName']), axis=1)
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory,AllocationFlow_CFS,AllocationVolume_AF,in_WaterSourceName,in_WaterSourceNativeID
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0.0,AF,0,None,130.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only,,0.0,Mumm Reservoir,WaDENE_WS1
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0.0,AF,0,None,20.0,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only,,0.0,Bowman Spring Branch Reservoir No. 2-A,WaDENE_WS2
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,,0.0,Bowman Spring Branch Reservoir No. 3-E,WaDENE_WS3
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0.0,AF,0,None,75.0,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,,0.0,Bowman Spring Branch Reservoir No. 3-E,WaDENE_WS3
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0.0,AF,0,None,314.0,Jefferson,Little Blue,102702070205,1969-06-25,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120,Storage Use-only,,0.0,Buckley Creek Reservoir 3-C,WaDENE_WS4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12236,542,Active,IU,D-680,SENE S11-T11-R26W,12280,1,1,1A,None,Six Mile Canal,None,Platte River,23.8,CFS,10681,70,1666.0,Lincoln,Twin Platte,102001010408,1894-10-22,None,1899-09-01T00:00:00,None,Open,2020-09-15T00:00:00,Open,Natural Flow (Reminder),1.020010e+11,-100.250737,40.937344,Irrigation and Incidental Underground Storage,23.8,,Platte River,WaDENE_WS32
12237,13057,Active,IR,A-19389,NENW S17-T3-R13E,16174,1,1,1F,None,Pump,None,Long Branch,0.43,CFS,193,70,29.6,Richardson,Nemaha,102400080205,2015-08-28,2018-09-01T00:00:00,2015-12-21T00:00:00,None,Open,2019-04-23T00:00:00,Open,Natural Flow (Reminder),1.024001e+11,-95.987384,40.229258,Irrigation from Natural Stream,0.43,,Long Branch,WaDENE_WS179
12238,1043,Active,IR,A-1285,NW S7-T5-R35W,2380,7,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404,1913-04-28,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094,Irrigation from Natural Stream,2.67,,Frenchman River,WaDENE_WS532
12239,1043,Active,IR,A-1285,NW S7-T5-R35W,2382,1,1,1B,None,Oliver Bros. Canal,None,Frenchman River,2.67,CFS,1198,70,187.0,Hayes,Middle Republican,102500050404,1913-04-28,None,1913-07-17T00:00:00,None,Open,2020-03-24T00:00:00,Open,Natural Flow (Reminder),1.025001e+11,-101.338500,40.413094,Irrigation from Natural Stream,2.67,,Frenchman River,WaDENE_WS532


In [13]:
# Exporting output files.
df.to_csv('P_NebraskaMaster.csv', index=False)  # The output.